<a href="https://colab.research.google.com/github/tproffen/ORCSGirlsPython/blob/master/Numbers/ChaosAndFractals-Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/tproffen/ORCSGirlsPython/blob/master/Images/Logo.png?raw=1" width="10%" align="right" hpsace="50">

# Numbers and Chaos

## Activity: The Logistical Map

Let us play with  numbers and unlock the secrets of the logistical map. Please make sure you run the cell with the setup commands below before running any other code in this notebook.


In [1]:
import ipywidgets as widgets
import plotly.graph_objects as go
import numpy as np

from google.colab import output
output.enable_custom_widget_manager()

Below is the plotting code which will eventually be in a helpers file and not visible. So do not worry if you do not understand the details :)

In [2]:
def makeInteractivePlot(func,p,plabel,pmin,pmax,xtitle='x',ytitle='y',mode='lines'):

  (xvals, yvals) = func(p)

  # Making the graph
  layout = go.Layout(yaxis_range=[min(yvals),max(yvals)], xaxis_title=xtitle, yaxis_title=ytitle,
                     width=800, height=600)
  data = go.Scatter(x=xvals, y=yvals, mode=mode, name='Line')
  graph = go.FigureWidget(data=[data], layout=layout)

  # update routine
  def update(change):
      (graph.data[0].x, graph.data[0].y) = func([s.value for s in pSliders])

  # Making the sliders
  pSliders=[]
  for i in range(len(p)):
    pSliders.append(widgets.FloatSlider(value=p[i], min=pmin[i], max=pmax[i], description=plabel[i]))
    pSliders[i].observe(update, names='value')

  sliders = widgets.VBox(pSliders)
  display(widgets.HBox([graph,sliders]))

### Simple plotting

First we look at `Plotly` and how to graph data. In the past we have used `matplotlib`, but here we want to create interactive plots in Google Colab, so we use `Plotly` instead. Let us start with a simple line. The equation for a line is simply

$y=m*x + b$

where $m$ is the slope and $b$ is the y-axis intercept. Because we want to easily recalculate the points of the line for plotting, we create a function that takes the slope $m$ and intercept $b$, the range we want to cover in x (as xmin, xmax and dx) and returns a list of x and y values.

In [3]:
def calculateLine(parameters):
  (m,b) = parameters                 # For our plotting, we pass all parameters for the
                                     # functions as a list. To make it easier to understand the
                                     # code we pass it to individual variables with the names
                                     # in the equation :)

  xmin = -5.0                        # Smallest x value to calculate the function
  xmax =  5.0                        # Largest x value to calculate the function
  dx = 0.05                          # Step size in x

  xvals = []                         # Empty list for x values
  yvals = []                         # Empty list for y values

  for x in np.arange(xmin,xmax,dx):  # Loop over points in x

      y = m*x + b                    # Calculate the y-value

      xvals.append(x)                # Add x valaue to the list
      yvals.append(y)                # Add y valaue to the list

  return xvals,yvals                 # Return the values

Time to plot.

In [4]:
makeInteractivePlot(calculateLine,[2.,0.],['m','b'],[-5,-5],[5,5], ytitle='y=ma+b')

    'data': [{'mode': 'lines',
              'name': 'Line',
              'type…

# Now the fun - the Logistic Map

You have heard about the logisicatl map in the class or you can watch <a href="https://youtu.be/ovJcsL7vyrk">this awesome video</a>.

Here is the equation:
$x_{n+1} = r x_{n}(1 - x_{n})$

Following the notation used in class, $r$ stands for the growth rate and $x$ is population. This is not the actual number, but the fraction of the maximum possible population, so it ranges from 0 to 1.

#### Step 1: Visualize the function

Let's define a function `growth` and plot how the population next year depends on the population this year;

In [5]:
def calculateGrowth(parameters):
  r = parameters[0]                  # Storing the parameter r from list

  xmin = 0.0                         # Smallest x value to calculate the function
  xmax = 1.0                         # Largest x value to calculate the function
  dx = 0.01                          # Step size in x

  xvals = []                         # Empty list for x values
  yvals = []                         # Empty list for y values

  for x in np.arange(xmin,xmax,dx):  # Loop over points in x

      y = r*x*(1-x)                  # Calculate the y-value

      xvals.append(x)                # Add x valaue to the list
      yvals.append(y)                # Add y valaue to the list

  return xvals,yvals                 # Return the values

In [6]:
makeInteractivePlot(calculateGrowth,[2.0],['r'],[0],[3], xtitle='x_n', ytitle='x_n+1')

    'data': [{'mode': 'lines',
              'name': 'Line',
              'type…

Woohoo, just like in the video. He have a 'single hump function' with a negative feedback, meaning when the population gets too big the dowturn of the parabula will reduce the population as you can see on the graph.

#### Step 2: Behaviour as time goes on

We are most interested in the long term behaviour of the function as we repeatley call it. In other words what happens to the population after a long time. In the function below we calculate the value of $x_{n} = r x_{n-1}(1 - x_{n-1})$ for all $n$. Note that we have rewritten this so the value we calculate os $n$ and it depends on the earlier value $n-1$ which is how we implement it by using the list element `[-1]`.

In [7]:
def calculateGrowthTime(parameters):
  (x0, r,nmax) = parameters                 # Storing the parameter from list
  nmax = int(nmax)

  # First point
  yvals=[x0]
  xvals=[0]

  for i in range(1,nmax):                    # Loop to the maximum number nmax we want.
    xvals.append(i)
    yvals.append(r*yvals[-1]*(1-yvals[-1]))  # Append the calculated value

  return xvals,yvals                         # Return all values

In [8]:
makeInteractivePlot(calculateGrowthTime,[0.4,1.2,200],['x0','r','nmax'],[0.1,0.5,20],[0.9,4.0,500], xtitle='Time', ytitle='Population', mode='lines+markers')

    'data': [{'mode': 'lines+markers',
              'name': 'Line',
           …

Explore how the function depends on `x0` and `r`. Anything spooky 😨

#### Step 3: Exploring as function of growth rate $r$

You might have seen strange behaviour for certain values of $r$, so next we explore the resulting populations after some time. Because there are multiple values the function oscillates around, we simply loop ofer $r$ and then plot the last `nplot` points ion a scatter plot.

In [20]:
x0 = 0.4         # Starting population
nmax  = 300      # Maximum value of n for calculate
nplot = 150      # First point to plot, so we plot nplot:nmax
rmin  = 2.9
rmax  = 4.0
dr = (rmax-rmin) / 1000.  # 1000 points

rvals=[]         # List for values of r
lvals=[]         # List for resulting values of x (there will be nmax values for each r)

for r in np.arange(rmin,rmax,dr):              # Loop over r - note the fine grid
    lvals.append(calculateGrowthTime([x0,r,nmax])[1][nplot:]) # Getting list of x starting at nplot
    rvals.append([r]*(nmax-nplot))              # We need the same number of r's so
                                                # lists are the same length

# Now we need to turn the list of lists into one contineous list for plotting
lvals = np.array(lvals).ravel()
rvals = np.array(rvals).ravel()

In [21]:
print(rvals)

[2.9    2.9    2.9    ... 3.9989 3.9989 3.9989]


All that is left is to plot. Adjust marker size and color below. Also explore changing nmax, nplot and the step size in r. Enjoy.

In [22]:
layout = go.Layout(xaxis_title='Growth rate r', yaxis_title='Population limit', width=800, height=600)
data = go.Scattergl(x=rvals, y=lvals, mode='markers', marker=dict(size=1, color='red'))
graph = go.FigureWidget(data=[data], layout=layout)

In [23]:
display(graph)

FigureWidget({
    'data': [{'marker': {'color': 'red', 'size': 1},
              'mode': 'markers',
              'type': 'scattergl',
              'uid': '097914a5-7564-4a94-84d9-7b4feca1c7a5',
              'x': array([2.9   , 2.9   , 2.9   , ..., 3.9989, 3.9989, 3.9989]),
              'y': array([0.65517241, 0.65517242, 0.65517241, ..., 0.22913317, 0.70633034,
                          0.82948299])}],
    'layout': {'height': 600,
               'template': '...',
               'width': 800,
               'xaxis': {'title': {'text': 'Growth rate r'}},
               'yaxis': {'title': {'text': 'Population limit'}}}
})

### How cool was that ..